# Artificial Neural Network

### Importing the libraries

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score


In [5]:
tf.__version__

'2.0.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [6]:
dataset = pd.read_csv('../data/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [7]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [8]:
print(y)

[1 0 1 ... 1 1 0]


#### Check prediction class 
We have a very imbalanced dataset, so we need to be carefull on the evaluation metrics we are going to use. In this case accuracy is not a valid measure as the accuracy of the model will be high by predicting the majority class correctly

In [11]:
dataset.iloc[:,-1].value_counts()/len(dataset)

0    0.7963
1    0.2037
Name: Exited, dtype: float64

### Encoding categorical data

Label Encoding the "Gender" column

In [6]:
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [8]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling
we fit the dcaler on the training data, and then apply it on th test data and on any other predictions we ant to make after the model is trained

In [11]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN
ANN are sequential, one layer after the other

In [12]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer
- When we want fully connected layer we use the Dense class where it initialised the input layer and the fully connected layer
- We just need to define the activation function, in this case relu and the number of nodes in the hiddel layer


In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer
not really needed, we got almost the same performance with just one hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer
- Here we need to define the activation agaon and we used sigmoid to transform the output to a probability 
- the dimensions of the output layer is one as the prediction is going to be a probability value, the treshold of which we could define as we wish. By default if probability > 0.5 then churner 

In [14]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN
- Stohastic gradient descent, with Adam optimiser
- we are optimisng for precision here as we want to predict as many correct churners as possible

In [15]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['Precision'])

### Training the ANN on the Training set
- We are performing batch optimisation, the weights are updated after the whole batch is processed

In [16]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Train on 8000 samples
Epoch 1/100
8000/8000 [==============================] - 1s 101us/sample - loss: 0.8159 - Precision: 0.2393
Epoch 2/100
8000/8000 [==============================] - 0s 28us/sample - loss: 0.5857 - Precision: 0.4443
Epoch 3/100
8000/8000 [==============================] - 0s 30us/sample - loss: 0.5177 - Precision: 0.5993TA: 0s - loss: 0.5280 - Precision: 0.5693
Epoch 4/100
8000/8000 [==============================] - 0s 30us/sample - loss: 0.4809 - Precision: 0.61570s - loss: 0.4861 - Precision: 0.61
Epoch 5/100
8000/8000 [==============================] - 0s 29us/sample - loss: 0.4573 - Precision: 0.6179
Epoch 6/100
8000/8000 [==============================] - 0s 29us/sample - loss: 0.4397 - Precision: 0.6479
Epoch 7/100
8000/8000 [==============================] - 0s 31us/sample - loss: 0.4241 - Precision: 0.6762
Epoch 8/100
8000/8000 [==============================] - 0s 33us/sample - loss: 0.4099 - Precision: 0.6888
Epoch 9/100
8000/8000 [======================

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

In [17]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

### Predicting the Test set results

In [18]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5) # turn to boolean True/False
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [19]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))

[[1519   76]
 [ 201  204]]
0.8615
0.7285714285714285
